In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [44]:
df = pd.read_csv("hate_speech_dataset/labeled_data.csv")

In [45]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [46]:
df.shape

(24783, 7)

In [47]:
df.describe()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000


In [48]:
df.isnull().sum()

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [49]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

def tokenize_and_encode(example):
    encoding = tokenizer(
        example["tweet"],               # or "text" if that's your column name
        truncation=True,
        padding="max_length",
        max_length=128
    )
    encoding["labels"] = example["class"]  # <- Add this line
    return encoding


In [50]:
df = df.drop(["Unnamed: 0","count", "hate_speech", "offensive_language", "neither"], axis=1)

In [51]:
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [52]:
df.shape

(24783, 2)

In [53]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)
tokenized_dataset = dataset.map(tokenize_and_encode)

Map: 100%|██████████| 4957/4957 [00:01<00:00, 2644.37 examples/s]


In [54]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilroberta-base",
    num_labels=3
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1 score": f1_score(labels, preds, average="weighted")
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\Acer\AppData\Local\Temp\ipykernel_15812\4048309274.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [56]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 score
1,0.317800,0.333895,0.908614,0.883057
2,0.256600,0.299637,0.921727,0.911660
3,0.229200,0.324033,0.917490,0.912274


TrainOutput(global_step=7437, training_loss=0.2810640559492051, metrics={'train_runtime': 1070.562, 'train_samples_per_second': 55.558, 'train_steps_per_second': 6.947, 'total_flos': 1969759111546368.0, 'train_loss': 0.2810640559492051, 'epoch': 3.0})

In [57]:
trainer.evaluate()
model.save_pretrained("hate-speech-distilroberta")
tokenizer.save_pretrained("hate-speech-distilroberta")

('hate-speech-distilroberta\\tokenizer_config.json',
 'hate-speech-distilroberta\\special_tokens_map.json',
 'hate-speech-distilroberta\\vocab.json',
 'hate-speech-distilroberta\\merges.txt',
 'hate-speech-distilroberta\\added_tokens.json',
 'hate-speech-distilroberta\\tokenizer.json')

In [65]:
import torch
from transformers import AutoTokenizer

# Load tokenizer (same one used during training)
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

# Set model to evaluation mode
model.eval()

# Define your label mapping if applicable
id2label = {0: "hate", 1: "offensive", 2: "not hate"}

# Input text to classify
text = "Kill that faggot"  # <-- change this to test different texts

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Move inputs and model to the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Perform prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# Print the result
print(f"Predicted class ID: {predicted_class}")
print(f"Predicted label: {id2label[predicted_class]}")


Predicted class ID: 0
Predicted label: hate
